# 0. The problem

Source: https://www.kaggle.com/competitions/rossmann-store-sales/overview

## Context

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

## Solution

Predict 6 weeks of daily sales for 1115 stores located across Germany. Reliable sales forecasts enable store managers to create effective staff schedules that increase productivity and motivation. By helping Rossmann create a robust prediction model, you will help store managers stay focused on what’s most important to them: their customers and their teams! 

# 1. Data description

**Files**
- train.csv - historical data including Sales
- test.csv - historical data excluding Sales
- store.csv - supplemental information about the stores

**Data fields**
- Id - an Id that represents a (Store, Date) duple within the test set
- Store - a unique Id for each store
- Sales - the turnover for any given day (this is what you are predicting)
- Customers - the number of customers on a given day
- Open - an indicator for whether the store was open: 0 = closed, 1 = open
- StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
- SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
- StoreType - differentiates between 4 different store models: a, b, c, d
- Assortment - describes an assortment level: a = basic, b = extra, c = extended
- CompetitionDistance - distance in meters to the nearest competitor store
- CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
- Promo - indicates whether a store is running a promo on that day
- Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
- Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
- PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

## 1.1. Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_theme(style="whitegrid")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from pathlib import Path

## 1.2. Loading data

In [ ]:
FOLDER_PATH = Path('../data/raw')

In [ ]:
df_store = pd.read_csv(filepath_or_buffer=FOLDER_PATH / "store.csv")
df_train_raw = pd.read_csv(filepath_or_buffer=FOLDER_PATH / "train.csv", low_memory=False)

# 2. Exploratory data analisys

## 2.1. Data description

### 2.1.1. Store

In [ ]:
# Information of each store
df_store.info()

In [ ]:
# Is there any duplicated store?
assert df_store.duplicated(subset=['Store']).sum() == 0

In [ ]:
df_store.sample(10)

In [ ]:
df_store["PromoInterval"].value_counts()

In [ ]:
# When a store is not participating of a continuing promotion, "Promo2SinceWeek", "Promo2SinceYear" and "PromoInterval" have NaN
df_store[["Promo2", "Promo2SinceWeek", "Promo2SinceYear", "PromoInterval"]]

In [ ]:
df_store.groupby("Promo2")["Promo2SinceWeek"].unique()

In [ ]:
df_store.groupby("Promo2")["Promo2SinceYear"].unique()

In [ ]:
df_store.groupby("Promo2")["PromoInterval"].unique()

**OBS.:**
- StoreType: change to category
- Assortment: change to category
- CompetitionOpenSinceMonth: change to int and add instance month when is NaN
- CompetitionOpenSinceYear: change to int and add instance year when is NaN
- Promo2SinceWeek: fill NaN with zero; change to int
- Promo2SinceYear: fill NaN with zero; change to int
- PromoInterval: fill NaN with zero; there are three intervals, change to a categorical value

StoreType

In [ ]:
df_store["StoreType"].value_counts()

Assortment

In [ ]:
assortment_map = {"a": "basic",
                  "b": "extra",
                  "c": "extended"}

In [ ]:
df_store["Assortment"] = df_store["Assortment"].map(assortment_map)

In [ ]:
df_store["Assortment"].value_counts()

In [ ]:
df_store["Assortment"].isna().sum()

CompetitionDistance
- NaN values could be related to a store with no competitors nearby
- Replace NaN with the max distance

In [ ]:
df_store[df_store["CompetitionDistance"].isna()]

In [ ]:
df_store["CompetitionDistance"].describe()

In [ ]:
sns.boxplot(data=df_store, x=df_store["CompetitionDistance"])

In [ ]:
df_store["CompetitionDistance"] = df_store["CompetitionDistance"].fillna(value=df_store["CompetitionDistance"].max())

In [ ]:
df_store["CompetitionDistance"].isna().sum()

### 2.1.2. Train

In [ ]:
df_train_raw.info()

Date

In [ ]:
# Change Date type to datetime
df_train_raw["Date"] = pd.to_datetime(df_train_raw["Date"])

In [ ]:
df_train_raw

StateHoliday

In [ ]:
holidays = {"a": "public",
            "b": "easter",
            "c": "xmas",
            "0": "none"}

df_train_raw.loc[:, "StateHoliday"] = df_train_raw["StateHoliday"].map(holidays)

In [ ]:
df_train_raw["StateHoliday"].value_counts()

In [ ]:
df_train_raw.groupby("StateHoliday")["Open"].value_counts(normalize=True)

In [ ]:
# There should be no repeated dates for each store
assert df_train_raw.duplicated(subset=["Store", "Date"]).sum() == 0

In [ ]:
# Number of instances for each store
df_train_raw.groupby("Store").size().value_counts()

In [ ]:
# Closed store has no sales, so it is not relevant
original_shape = df_train_raw.shape
df_train_raw = df_train_raw[df_train_raw["Open"] == 1]
df_train_raw = df_train_raw.drop(columns=["Open"])
print(f"Original shape: {original_shape}")
print(f"New shape: {df_train_raw.shape}")
print(f"Removed lines: {original_shape[0] - df_train_raw.shape[0]}")

### 2.1.3. Join train and store dataframes

In [ ]:
df_raw = pd.merge(df_train_raw, df_store, how="left", on="Store")
df_raw.shape

## 2.2. Data analisys

CompetitionOpenSinceMonth
- When there is no month, the month of the column Date will be used
- Doing this I am saing the competitor has opened at the some day of the registered sale

In [ ]:
df_raw["CompetitionOpenSinceMonth"].isna().sum()

In [ ]:
df_raw["CompetitionOpenSinceMonth"] = df_raw["CompetitionOpenSinceMonth"].fillna(value=df_raw["Date"].dt.month)
assert df_raw["CompetitionOpenSinceMonth"].isna().sum() == 0

CompetitionOpenSinceYear
- Same approach from the Month

In [ ]:
df_raw["CompetitionOpenSinceYear"].isna().sum()

In [ ]:
df_raw["CompetitionOpenSinceYear"] = df_raw["CompetitionOpenSinceYear"].fillna(value=df_raw["Date"].dt.year)
assert df_raw["CompetitionOpenSinceYear"].isna().sum() == 0

Promo2SinceWeek and Promo2SinceYear
- All the missing values occour when Promo2 is zero
- Replace with zero, too

In [ ]:
df_raw.groupby("Promo2")["Promo2SinceWeek"].unique()

In [ ]:
df_raw.groupby("Promo2")["Promo2SinceYear"].unique()

In [ ]:
df_raw.loc[:, "Promo2SinceWeek"] = df_raw["Promo2SinceWeek"].fillna(value=0)
df_raw.loc[:, "Promo2SinceYear"] = df_raw["Promo2SinceYear"].fillna(value=0)

In [ ]:
assert df_raw["Promo2SinceWeek"].isna().sum() == 0
assert df_raw["Promo2SinceYear"].isna().sum() == 0

PromoInterval
- Replace NaN with zero
- Map three intervals corresponding the three groups of month

In [ ]:
df_raw.groupby("Promo2")["PromoInterval"].unique()

In [ ]:
df_raw["PromoInterval"].unique()

In [ ]:
df_raw.loc[:, "PromoInterval"] = df_raw["PromoInterval"].fillna(value=0)

In [ ]:
promo_intervals = {0: 0,
                   "Jan,Apr,Jul,Oct": 1,
                   "Feb,May,Aug,Nov": 2,
                   "Mar,Jun,Sept,Dec": 3}

df_raw.loc[:, "PromoInterval"] = df_raw["PromoInterval"].map(promo_intervals)

In [ ]:
assert df_raw["PromoInterval"].isna().sum() == 0

Changing types

In [ ]:
# Changing types
df_raw["CompetitionOpenSinceMonth"] = df_raw["CompetitionOpenSinceMonth"].astype(int)
df_raw["CompetitionOpenSinceYear"] = df_raw["CompetitionOpenSinceYear"].astype(int)
df_raw["Promo2SinceWeek"] = df_raw["Promo2SinceWeek"].astype(int)
df_raw["Promo2SinceYear"] = df_raw["Promo2SinceYear"].astype(int)

In [ ]:
df_raw.info()

In [ ]:
num_attributes = ['Store', 'DayOfWeek', 'Sales', 'Customers', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
                  'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear']

In [ ]:
df_raw[num_attributes].describe().T

In [ ]:
sns.boxplot(df_raw["CompetitionOpenSinceYear"], orient="h")

In [ ]:
# Rossmann company was founded in 1972.
# Replace year 1900 by the next min year.
comp_open_since_y = df_raw[df_raw["CompetitionOpenSinceYear"] != 1900]["CompetitionOpenSinceYear"].min()
df_raw.loc[df_raw["CompetitionOpenSinceYear"] == 1900, "CompetitionOpenSinceYear"] = comp_open_since_y

In [ ]:
df_raw[num_attributes].describe().T

Sales

In [ ]:
sns.histplot(data=df_raw, x="Sales", bins=30)

Ex: Store 1 daily Sales

In [ ]:
store_n = 1
df_raw_store = df_raw[df_raw["Store"] == store_n][["Date", "Sales"]]
fig, ax = plt.subplots(figsize=(12,5))
sns.lineplot(data=df_raw_store, x="Date", y="Sales")
plt.title(f"Store {store_n} daily sales")

**Hypothesis:**

**1.** Stores with extended assortment should sell more. **FALSE**

In [ ]:
sns.barplot(df_raw.groupby("Assortment")["Sales"].sum())

**2.** Stores with closer competitors should sell less. **FALSE**

In [ ]:
hyp2 = df_raw[["CompetitionDistance", "Assortment", "Sales"]]\
    .groupby(["CompetitionDistance", "Assortment"])["Sales"].sum().reset_index()
hyp2

In [ ]:
sns.scatterplot(data=hyp2, x="CompetitionDistance", y="Sales", hue="Assortment")

**3.** Stores with long-standing competitors should sell more. **FALSE**

In [ ]:
max_year_sales = df_raw.groupby(["Store"])["Date"].max().dt.year.reset_index()
max_year_sales.rename(columns={"Date": "CurrentYear"}, inplace=True)

In [ ]:
hyp3 = df_raw[["Store", "CompetitionOpenSinceYear", "Sales"]]\
    .groupby(["Store", "CompetitionOpenSinceYear"])["Sales"].sum().reset_index()

In [ ]:
hyp3 = pd.merge(hyp3, max_year_sales, on="Store", how="left")
hyp3["CompetitionOpenedWeeks"] = (hyp3["CurrentYear"] - hyp3["CompetitionOpenSinceYear"]) * 52

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
sns.barplot(hyp3.groupby("CompetitionOpenedWeeks")["Sales"].sum(), ax=ax)
plt.title("Total store sales based on time since the nearest competitor opened")
plt.ylabel("Total Sales")
plt.xticks(rotation=45);

**4.** Stores opened in Christmas holiday shoul sell more. **FALSE**

In [ ]:
df_raw.groupby("StateHoliday")["Sales"].sum()

In [ ]:
hyp4a = df_raw.query("StateHoliday != 'none'").groupby("StateHoliday")["Sales"].sum().reset_index().sort_values(by="Sales")

In [ ]:
hyp4b = df_raw[["Date", "StateHoliday", "Sales"]].groupby([df_raw["Date"].dt.year, "StateHoliday"])["Sales"].sum().reset_index()
hyp4b = hyp4b[hyp4b["StateHoliday"] != "none"]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,5))
sns.barplot(data=hyp4a, x="StateHoliday", y="Sales", ax=ax[0])
sns.barplot(data=hyp4b, x="Date", y="Sales", hue="StateHoliday", ax=ax[1])

**5.** Stores should sell more over the years. **IN TERMS OF ABSOLUTE VALUES IS FALSE**

In [ ]:
hyp5 = df_raw[["Date", "Sales"]].copy()
hyp5["Year"] = hyp5["Date"].dt.year
hyp5.drop(columns="Date", inplace=True)
hyp5

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16 ,5))
sns.barplot(data=hyp5, x="Year", y="Sales", estimator="mean", errorbar="sd", ax=ax[0])
sns.barplot(data=hyp5, x="Year", y="Sales", estimator="sum", errorbar="sd", ax=ax[1])
ax[0].set_title("Average sales over the years")
ax[1].set_title("Total sales over the years")
ax[0].set_xlabel("Year")
ax[1].set_xlabel("Year")
ax[0].set_ylabel("Average Sales")
ax[1].set_ylabel("Total Sales")

**6.** Stores should sell more in the second half of the year. **FALSE**

In [ ]:
hyp6a = df_raw.groupby([df_raw["Date"].dt.year, df_raw["Date"].dt.month])[["Sales"]].sum()
hyp6a.index.names = ["Year", "Month"]
hyp6a = hyp6a.reset_index()

In [ ]:
fig = sns.FacetGrid(data=hyp6a, col="Year", sharey=True, height=5)
fig.map(sns.barplot, "Month", "Sales", order=sorted(hyp6a["Month"].unique()))

In [ ]:
hyp6a["Sem"] = hyp6a["Month"].apply(lambda x: 1 if x <= 6 else 2)

In [ ]:
sns.barplot(data=hyp6a.groupby(["Year", "Sem"])["Sales"].sum().reset_index(), x="Sem", y="Sales", hue="Year")

**7.** Stores should sell less on weekends. **FALSE**

In [ ]:
sns.barplot(data=df_raw.groupby("DayOfWeek")["Sales"].mean().reset_index(), x="DayOfWeek", y="Sales")
plt.title("Daily total sales")
plt.xlabel("Day")
plt.ylabel("Total Sales")
plt.xticks(ticks=[0,1,2,3,4,5,6], labels=["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]);

**8.** Stores should sell less during school holidays. **TRUE FOR TOTAL SALES**

In [ ]:
sns.barplot(data=df_raw.groupby("SchoolHoliday")["Sales"].sum().reset_index(), x="SchoolHoliday", y="Sales")
plt.title("Daily total sales")
plt.xlabel("School holiday")
plt.ylabel("Total Sales");

# 3. Data preparation

## 3.1. Changing data types

In [ ]:
df_raw["PromoInterval"] = df_raw["PromoInterval"].astype(dtype="category")
df_raw["Promo"] = df_raw["Promo"].astype(dtype="category")
df_raw["SchoolHoliday"] = df_raw["SchoolHoliday"].astype("category")
df_raw["Promo2"] = df_raw["Promo2"].astype("category")

## 3.2. Feature creation

In [ ]:
def CreateLagFeature(df):
    for lag in [7, 14, 28]:
        df[f"Sales_lag_{lag}"] = df.sort_values(by=["Store", "Date"]).groupby("Store")["Sales"].shift(lag)
    return df

In [ ]:
def CreateRollingMeanFeature(df):
    for window in [7, 14, 28]:
        df[f"Sales_roll_mean_{window}"] = df.sort_values(by=["Store", "Date"])["Sales"].rolling(window=window, closed="left").mean()
    return df

In [ ]:
def FeatureCreation(df):
    df["DayOfYear"] = df["Date"].dt.day_of_year
    df["Year"] = df["Date"].dt.year
    df["QuarterOfYear"] = df["Date"].dt.quarter
    df["Month"] = df["Date"].dt.month
    df["WeekOfYear"] = df["Date"].dt.isocalendar().week
    df = CreateLagFeature(df)
    df = CreateRollingMeanFeature(df)
    df.dropna(inplace=True)
    return df

In [ ]:
df_raw = FeatureCreation(df_raw)

In [ ]:
store_n = 1
year = 2014
df_raw_store = df_raw[(df_raw["Store"] == store_n) & (df_raw["Date"].dt.year == year)][["Date", "Sales"]]
fig, ax = plt.subplots(figsize=(12,5))
sns.lineplot(data=df_raw_store, x="Date", y="Sales", label="Train set")
plt.title(f"Store {store_n} - one year sales")

In [ ]:
store_n = 1
df_raw_store = df_raw[df_raw["Store"] == store_n][["Date", "Sales"]]
fig, ax = plt.subplots(figsize=(12,5))
sns.lineplot(data=df_raw_store, x="Date", y="Sales", label="Train set")
plt.title(f"Store {store_n} - daily sales")

## 3.3. Encoding

### StateHoliday

In [ ]:
ohe_stt_hld = OneHotEncoder(sparse_output=False, dtype="int").set_output(transform="pandas")
stt_hld = ohe_stt_hld.fit_transform(df_raw[["StateHoliday"]])
df_raw = pd.concat(objs=[df_raw, stt_hld], axis="columns").drop(columns="StateHoliday")

### StoreType

In [ ]:
ohe_sto_typ = OneHotEncoder(sparse_output=False, dtype="int").set_output(transform="pandas")
sto_typ = ohe_sto_typ.fit_transform(df_raw[["StoreType"]])
df_raw = pd.concat(objs=[df_raw, sto_typ], axis="columns").drop(columns="StoreType")

### Assortment

In [ ]:
ohe_assort = OneHotEncoder(sparse_output=False, dtype="int").set_output(transform="pandas")
assort = ohe_assort.fit_transform(df_raw[["Assortment"]])
df_raw = pd.concat(objs=[df_raw, assort], axis="columns").drop(columns="Assortment")

## 3.4. Scaling

In [ ]:
lag_rolling_features = [col for col in df_raw.columns if "Sales_lag" in col or "Sales_roll" in col]

In [ ]:
std_scaler = StandardScaler()
df_raw[lag_rolling_features] = std_scaler.fit_transform(df_raw[lag_rolling_features])

## 3.5. Train-test split

- It is necessary to set aside the last 6 registered weeks of each store for testing

In [ ]:
def split_store_data(df, test_size=6*7):
    stores = df["Store"].unique()
    train_list = []
    test_list = []

    for store in stores:
        store_data = df[df["Store"] == store].sort_values(by="Date")
        train, test = train_test_split(store_data, test_size=test_size, shuffle=False)
        train_list.append(train)
        test_list.append(test)

    df_train = pd.concat(train_list)
    df_test = pd.concat(test_list)
    
    return df_train, df_test

In [ ]:
df_train, df_test = split_store_data(df=df_raw, test_size=6*7)

In [ ]:
df_train.shape, df_test.shape

In [ ]:
# Checking data splitting
assert df_train.shape[0] + df_test.shape[0] == df_raw.shape[0], "Problem in data splitting"

In [ ]:
stores = df_store["Store"].unique()
for store in stores:
    assert df_train[df_train["Store"] == store]["Date"].max() < df_test[df_test["Store"] == store]["Date"].min(), \
        "Minimum test data is earlier than maximum train data"

In [ ]:
df_test.columns

## 3.6. Saving processed data

In [ ]:
SAVING_PATH = Path("../data/processed")

df_train.to_csv(SAVING_PATH / "train.csv", index=False)
df_test.to_csv(SAVING_PATH / "test.csv", index=False)

In [ ]:
store_n = 1
df_train_store = df_train[df_train["Store"] == store_n][["Date", "Sales"]]
df_test_store = df_test[df_test["Store"] == store_n][["Date", "Sales"]]
fig, ax = plt.subplots(figsize=(12,5))
sns.lineplot(data=df_train_store, x="Date", y="Sales", label="Train set")
sns.lineplot(data=df_test_store, x="Date", y="Sales", label="Test set")
plt.title(f"Store {store_n} daily sales")